# DENTIST APPOINTMENT MANAGEMENT SYSTEM PHYSICAL IMPLEMENTATION

In [1]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [2]:
def connect():
    connection = psycopg2.connect(user="postgres",
                                  password="admin",
                                  host="localhost",
                                  port="5432",
                                  )
    connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
    return connection

### CREATING DATABASE "dentist_appointment_management"

In [5]:

try:
    connection = connect()
    query = "CREATE DATABASE dentist_appointment_management;"
    
    
    cursor = connection.cursor()
    
    # Executing a SQL query
    cursor.execute(query)
    print("Database sucessfully created")
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print('')
        print("PostgreSQL connection is closed")

Database sucessfully created

PostgreSQL connection is closed


#### Defining connect() method so we can use it laster through out our code

In [2]:
def connect():
    connection = psycopg2.connect(user="postgres",
                                  password="admin",
                                  host="localhost",
                                  port="5432",
                                  database="dentist_appointment_management")
    
    return connection

### Creating tables in the database dentist_appointment_management

In [100]:
import logging
try:
    connection = connect()
    
    cursor = connection.cursor()
    
    query_1 = '''
    CREATE TABLE country (
    id SERIAL PRIMARY KEY,
    country_name TEXT
    )
    '''
    cursor.execute(query_1)
    
    query_2 = '''
    CREATE TABLE state (
    id SERIAL PRIMARY KEY,
    state_name TEXT,
    country_id INTEGER,
    
    CONSTRAINT fk_state_country
    FOREIGN KEY(country_id) REFERENCES country(id)
    )
    '''
    cursor.execute(query_2)
    
    query_3 = '''
    CREATE TABLE city (
    id SERIAL PRIMARY KEY,
    city_name TEXT,
    state_id INTEGER,
    
    CONSTRAINT fk_city_state
    FOREIGN KEY(state_id) REFERENCES state(id)
    )
    '''
    cursor.execute(query_3)
    
    query_4 = '''
    CREATE TABLE place (
    id SERIAL PRIMARY KEY,
    place_name TEXT,
    city_id INTEGER,
    
    CONSTRAINT fk_street_city
    FOREIGN KEY(city_id) REFERENCES city(id)
    )
    '''
    cursor.execute(query_4)
    
    query_5 = '''
    CREATE TABLE patient (
    patient_number INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    phone TEXT UNIQUE NOT NULL,
    email TEXT UNIQUE NOT NULL,
    address_id INTEGER,
    
    CONSTRAINT fk_patient_place
    FOREIGN KEY(address_id) REFERENCES place(id)
    
    )
    '''
    cursor.execute(query_5)
    
    query_6 ='''
    CREATE TABLE dentist (
    id SMALLINT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    phone TEXT UNIQUE NOT NULL,
    email TEXT UNIQUE NOT NULL,
    photo_path TEXT
    )
    '''
    cursor.execute(query_6)
    
    query_7 = '''
    CREATE TABLE dental_office (
    id CHAR PRIMARY KEY,
    name TEXT,
    address TEXT,
    phone TEXT UNIQUE NOT NULL
    )
    '''
    cursor.execute(query_7)
    
    
    query_8 = '''
    CREATE TABLE availability (
    date DATE,
    dentist_id SMALLINT,
    dental_office_id CHAR,
    
    CONSTRAINT fk_availability_dentist
    FOREIGN KEY(dentist_id) REFERENCES dentist(id),
    
    CONSTRAINT fk_availability_dental_office 
    FOREIGN KEY(dental_office_id) REFERENCES dental_office(id),
    
    PRIMARY KEY(date,dentist_id,dental_office_id),
    
    UNIQUE(date,dentist_id)
    )
    '''
    cursor.execute(query_8)
    
    
    query_9 = '''
    CREATE TABLE dental_hygenist (
    id SMALLINT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    phone TEXT UNIQUE NOT NULL,
    dental_office_id CHAR,
    photo_path TEXT,
    
    CONSTRAINT fk_dental_hygenist_dental_office
    FOREIGN KEY(dental_office_id) REFERENCES dental_office(id)
    
    
    )
    '''
    cursor.execute(query_9)
    

    
    query_10 = '''
    CREATE TABLE appointment(
    id SERIAL PRIMARY KEY,
    patient_id INTEGER,
    type TEXT,
    datetime_of_appointment TIMESTAMP,
    dentist_id SMALLINT,
    
    CONSTRAINT fk_appointment_patient
    FOREIGN KEY(patient_id) REFERENCES patient(patient_number),
    
    CONSTRAINT fk_appointment_dentist
    FOREIGN KEY(dentist_id) REFERENCES dentist(id),
    
    CONSTRAINT unique_datetime_of_appointment_dentist 
    UNIQUE (datetime_of_appointment , dentist_id)
    
    )
    '''
    cursor.execute(query_10)
    
    query_11 = '''
    CREATE TABLE operation(
    appointment_id INTEGER PRIMARY KEY,
    dental_hygenist_id SMALLINT,
    dentist_id SMALLINT,
    
    CONSTRAINT fk_operation_dental_hygenist
    FOREIGN KEY(dental_hygenist_id) REFERENCES dental_hygenist(id),
    
    CONSTRAINT fk_operation_appointment
    FOREIGN KEY(appointment_id) REFERENCES appointment(id),
    
    CONSTRAINT fk_operation_dentist
    FOREIGN KEY(dentist_id) REFERENCES dentist(id)
    )
    '''
    cursor.execute(query_11)
    
    query_12 = '''
    CREATE TABLE invoice(
    id INTEGER PRIMARY KEY,
    patient_id INTEGER,
    operation_id INTEGER,
    datetime TIMESTAMP,
    amount MONEY,
    
    CONSTRAINT fk_invoice_patient
    FOREIGN KEY(patient_id) REFERENCES patient(patient_number),
    
    CONSTRAINT fk_invoice_operation
    FOREIGN KEY(operation_id) REFERENCES operation(appointment_id)
    )
    '''
    cursor.execute(query_12)
    
    
    connection.commit()
    
    
    
    print("Tables sucessfully created")
except (Exception,psycopg2.Error) as error:
    #print("Error while connecting to PostgresSQL: ",error)
    #print(error)
    logging.debug("An exception was thrown!", exc_info=True)
    logging.info("An exception was thrown!", exc_info=True)
    logging.warning("An exception was thrown!", exc_info=True)
finally:
    if(connection):
        cursor.close()
        connection.close()
        print('')
        print("PostgreSQL connection is closed")

Tables sucessfully created

PostgreSQL connection is closed


## Populating data to "dentist_appointment_management" database from .xlsx file.

####  file link: https://docs.google.com/spreadsheets/d/1lq8rIx7LCa7BAixCdN4ZItkLpHoLKaw5a632lyiH6DI/edit?usp=sharing

In [101]:
import pandas as pd
import logging

In [102]:
tables = ['country','state','city','place','dentist','patient','dental_office','availability','dental_hygenist','appointment','operation','invoice']

In [103]:
try:
    connection=connect()
    cursor = connection.cursor()
    for table in tables:
        df = pd.read_excel ('dental appointment datasets.xlsx',sheet_name=table)
        for columns in df.columns:
            #print(columns)
            try:
                if df[columns].dtypes =='float':
                    print(str(columns))
                    df[columns]=df[columns].astype('int32')
            except:
                print(str(columns))
                pass

        data = [list(x) for x in df.itertuples(index=False)]
        query = "INSERT INTO " + str(table)+" VALUES(" +'%s'+',%s'*(len(df.columns)-1) + ');'

        print(query)
        for item in data:
            to_push=item
            for i,ele in enumerate(item):
                if str(ele)=='nan':
                    to_push[i]=None
                    
            cursor.execute(query,to_push)
            connection.commit()
        print("Data sucessufully inserted of table "+str(table))
        
except(Exception,psycopg2.Error) as e:
    print(e)
    logging.debug("An exception was thrown!", exc_info=True)
    logging.info("An exception was thrown!", exc_info=True)
    logging.warning("An exception was thrown!", exc_info=True)

finally:
    if(connection):
        cursor.close()
        connection.close()
        print('')
        print("PostgreSQL connection is closed")

INSERT INTO country VALUES(%s,%s);
Data sucessufully inserted of table country
INSERT INTO state VALUES(%s,%s,%s);
Data sucessufully inserted of table state
INSERT INTO city VALUES(%s,%s,%s);
Data sucessufully inserted of table city
INSERT INTO place VALUES(%s,%s,%s);
Data sucessufully inserted of table place
INSERT INTO dentist VALUES(%s,%s,%s,%s,%s,%s);
Data sucessufully inserted of table dentist
INSERT INTO patient VALUES(%s,%s,%s,%s,%s,%s);
Data sucessufully inserted of table patient
INSERT INTO dental_office VALUES(%s,%s,%s,%s);
Data sucessufully inserted of table dental_office
INSERT INTO availability VALUES(%s,%s,%s);
Data sucessufully inserted of table availability
INSERT INTO dental_hygenist VALUES(%s,%s,%s,%s,%s,%s);
Data sucessufully inserted of table dental_hygenist
INSERT INTO appointment VALUES(%s,%s,%s,%s,%s);
Data sucessufully inserted of table appointment
INSERT INTO operation VALUES(%s,%s,%s);
Data sucessufully inserted of table operation
INSERT INTO invoice VALUES(%s

## Queries

### Looking at the dental offices

In [104]:
try:
    connection = connect()
    cursor = connection.cursor()
    
    query = '''
    SELECT * FROM dental_office
    '''
    
    cursor.execute(query)
    col = [desc[0] for desc in cursor.description]  

    df=pd.DataFrame(columns=col,data=cursor.fetchall())
except(Exception,psycopg2.Error) as e:
    print(e)
    logging.debug("An exception was thrown!", exc_info=True)
    logging.info("An exception was thrown!", exc_info=True)
    logging.warning("An exception was thrown!", exc_info=True)

finally:
    if(connection):
        cursor.close()
        connection.close()
        print('')
        print("PostgreSQL connection is closed")
df


PostgreSQL connection is closed


,id,name,address,phone
0,A,Jawlakhel Office,"Jawlakhel,Kathmandu",98345345345
1,B,Putalisadak Office,"Putalisadak,Kathmandu",98346838113


### Looking at all the dental_hygenist related to specific dental office

In [105]:
try:
    connection = connect()
    cursor = connection.cursor()
    
    query = '''
    SELECT * FROM dental_office
    INNER JOIN dental_hygenist h
        ON dental_office.id = h.dental_office_id
    '''
    
    cursor.execute(query)
    col = [desc[0] for desc in cursor.description]  

    df=pd.DataFrame(columns=col,data=cursor.fetchall())
except(Exception,psycopg2.Error) as e:
    print(e)
    logging.debug("An exception was thrown!", exc_info=True)
    logging.info("An exception was thrown!", exc_info=True)
    logging.warning("An exception was thrown!", exc_info=True)

finally:
    if(connection):
        cursor.close()
        connection.close()
        print('')
        print("PostgreSQL connection is closed")
df


PostgreSQL connection is closed


,id,name,address,phone,id,first_name,last_name,phone,dental_office_id,photo_path
0,A,Jawlakhel Office,"Jawlakhel,Kathmandu",98345345345,1,Gita,Rai,9876547586,A,c:ddd/ytut
1,A,Jawlakhel Office,"Jawlakhel,Kathmandu",98345345345,2,Jina,Magar,9870674832,A,c:ddd/eee
2,B,Putalisadak Office,"Putalisadak,Kathmandu",98346838113,3,Sushant,Parajuli,9874747575,B,c:ddd/ytuee
3,B,Putalisadak Office,"Putalisadak,Kathmandu",98346838113,4,Ramu,Pandey,9874740489,B,c:ddd/ytuttt


### Looking at all the dentist

In [106]:
try:
    connection = connect()
    cursor = connection.cursor()
    
    query = '''
    SELECT * FROM dentist
    '''
    
    cursor.execute(query)
    col = [desc[0] for desc in cursor.description]  

    df=pd.DataFrame(columns=col,data=cursor.fetchall())
except(Exception,psycopg2.Error) as e:
    print(e)
    logging.debug("An exception was thrown!", exc_info=True)
    logging.info("An exception was thrown!", exc_info=True)
    logging.warning("An exception was thrown!", exc_info=True)

finally:
    if(connection):
        cursor.close()
        connection.close()
        print('')
        print("PostgreSQL connection is closed")
df


PostgreSQL connection is closed


,id,first_name,last_name,phone,email,photo_path
0,1,Shristy,Gautam,9876463526,shristy@gmail.com,c:/asdasd/asdasds
1,2,Nitisha,Timalsina,9836373663,nitisha@gmail.com,c:/asdasd/asdassd
2,3,Urmila,Acharya,9870784940,urmila@gmail.com,c:/asdasd/asdasssd
3,4,Rita,Thapa,9876575844,rita@gmail.com,c:/asdasd/asdasd33
4,5,Rubina,Singh,9846474784,rubina@gmail.com,c:/asdasd/asdafgsd


### Looking at all the total number of operations performed by dentist

In [133]:
try:
    connection = connect()
    cursor = connection.cursor()
    
    query = '''
    SELECT CONCAT(dentist.first_name,' ',dentist.last_name) AS "Dentist Name" , 
    count(*) AS " Total Operation Performed"
    FROM operation
    INNER JOIN dentist 
        ON operation.dentist_id = dentist.id
    GROUP BY CONCAT(dentist.first_name,' ',dentist.last_name)
    
    '''
    
    cursor.execute(query)
    col = [desc[0] for desc in cursor.description]  

    df=pd.DataFrame(columns=col,data=cursor.fetchall())
except(Exception,psycopg2.Error) as e:
    print(e)
    logging.debug("An exception was thrown!", exc_info=True)
    logging.info("An exception was thrown!", exc_info=True)
    logging.warning("An exception was thrown!", exc_info=True)

finally:
    if(connection):
        cursor.close()
        connection.close()
        print('')
        print("PostgreSQL connection is closed")
df


PostgreSQL connection is closed


,Dentist Name,Total Operation Performed
0,Urmila Acharya,4
1,Nitisha Timalsina,6
2,Rubina Singh,2
3,Rita Thapa,5
4,Shristy Gautam,9


### All the oppointment

In [108]:
try:
    connection = connect()
    cursor = connection.cursor()
    
    query = '''
    SELECT * FROM appointment
    '''
    
    cursor.execute(query)
    col = [desc[0] for desc in cursor.description]  

    df=pd.DataFrame(columns=col,data=cursor.fetchall())
except(Exception,psycopg2.Error) as e:
    print(e)
    logging.debug("An exception was thrown!", exc_info=True)
    logging.info("An exception was thrown!", exc_info=True)
    logging.warning("An exception was thrown!", exc_info=True)

finally:
    if(connection):
        cursor.close()
        connection.close()
        print('')
        print("PostgreSQL connection is closed")
df


PostgreSQL connection is closed


,id,patient_id,type,datetime_of_appointment,dentist_id
0,1,1,Teeth Cleaning,2021-09-08 10:00:00,1
1,2,2,Teeth Cleaning,2021-09-08 11:00:00,1
2,3,3,Teeth Cleaning,2021-09-08 12:00:00,1
3,4,4,Teeth Cleaning,2021-09-08 10:00:00,2
4,5,5,Teeth Cleaning,2021-09-08 11:00:00,2
5,6,6,Teeth Cleaning,2021-09-08 14:00:00,4
6,7,7,Teeth Cleaning,2021-09-08 13:00:00,5
7,8,5,Fillings and Restorations,2021-09-09 15:00:00,1
8,9,6,Tooth Extractions.,2021-09-09 12:00:00,1
9,10,7,Fillings and Restorations,2021-09-09 13:00:00,1


### ALL of the appointments in the future from NOW() and the respective Dentist and The dental office assigned to the patient

In [109]:
from datetime import datetime

In [110]:
datetime.now()

datetime.datetime(2021, 9, 12, 13, 7, 32, 70611)

In [129]:
try:
    connection = connect()
    cursor = connection.cursor()
    
    query = '''
    SELECT 
    CONCAT(p.first_name ,' ',p.last_name) AS "Patient Name",
    p.phone as "Patient Phone",
    CONCAT(d.first_name,'',d.last_name) AS "Dentist Name",
    a.datetime_of_appointment,
    a.type AS "Appointment issue",
    dental_office.name AS "Dental Office",
    dental_office.address AS "Dental Offfice Address"
    FROM appointment a
    INNER JOIN dentist d
        ON a.dentist_id = d.id
    INNER JOIN availability
        ON (DATE(a.datetime_of_appointment),a.dentist_id) = (availability.date,availability.dentist_id)
    LEFT JOIN patient p
        ON a.patient_id = p.patient_number
    INNER JOIN dental_office
        ON availability.dental_office_id = dental_office.id
    WHERE a.datetime_of_appointment > localtimestamp
    '''
    
    cursor.execute(query)
    col = [desc[0] for desc in cursor.description]  

    df=pd.DataFrame(columns=col,data=cursor.fetchall())
except(Exception,psycopg2.Error) as e:
    print(e)
    logging.debug("An exception was thrown!", exc_info=True)
    logging.info("An exception was thrown!", exc_info=True)
    logging.warning("An exception was thrown!", exc_info=True)

finally:
    if(connection):
        cursor.close()
        connection.close()
        print('')
        print("PostgreSQL connection is closed")
df


PostgreSQL connection is closed


,Patient Name,Patient Phone,Dentist Name,datetime_of_appointment,Appointment issue,Dental Office,Dental Offfice Address
0,paul mccartney,98767899097,RitaThapa,2021-09-13 10:00:00,Wisdom Teeth Removal,Putalisadak Office,"Putalisadak,Kathmandu"
1,Shreya Timalsina,98767899096,ShristyGautam,2021-09-14 10:00:00,Dental Sealants,Putalisadak Office,"Putalisadak,Kathmandu"
2,Nabin Gautam,98454535345,UrmilaAcharya,2021-09-15 10:00:00,Wisdom Teeth Removal,Putalisadak Office,"Putalisadak,Kathmandu"
3,Dipesh prasai,98435345334,RitaThapa,2021-09-16 10:00:00,Wisdom Teeth Removal,Jawlakhel Office,"Jawlakhel,Kathmandu"
